In [ ]:
!pip3 install tensorflow-gpu

In [177]:
from tensorflow.python.client import device_lib

In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.
/home/shiva_sync/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shiva_sync/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shiva_sync/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shiva_sync/.local/lib/python3.5/site-packages/t

[]

In [3]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6080954016251786921
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5078124165175639530
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16181178921913356226
physical_device_desc: "device: XLA_CPU device"
]


In [2]:
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [3]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

print( tf.VERSION )

import json
import gzip


filename = 'qa_Electronics.json.gz'  # Sample file.
questions=[]
answers=[]
i=0
with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:  # Read one line.
        i+=1
        l=eval(line)
        questions.append(l['question'])
        answers.append( '<START> ' + l['answer'] + ' <END>' )
        if i==10000:
            break
       
        

tokenizer = preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,)
tokenizer.fit_on_texts( questions + answers )
count_thres = 3
low_count_words = [w for w,c in tokenizer.word_counts.items() if c < count_thres]

for w in low_count_words:
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]

VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))  

1.14.0
VOCAB SIZE : 7154


In [4]:
import os
embeddings_index = {}
f = open(os.path.join('glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
max_features = 20000
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 200

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

Found 7153 unique tokens.
7154


In [ ]:
questions[1]

In [17]:
tokenized_questions = tokenizer.texts_to_sequences( questions )

In [19]:
len(tokenized_questions)

10000

In [6]:
from tensorflow.keras import preprocessing , utils

import os
import yaml
import numpy as np
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = 20
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = 20
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(10000, 20) 20
(10000, 20) 20
(10000, 20, 7154)


In [8]:
import keras
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Concatenate,Input
from keras.initializers import Constant
from keras.models import Model
encoder_inputs = Input(shape=( None , ))
encoder_embedding = Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix), mask_zero=True ) (encoder_inputs)
#encoder_outputs , state_h , state_c = LSTM( 200 , return_state=True )( encoder_embedding )

encoder = Bidirectional(LSTM(200, return_state=True,return_sequences=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_embedding)
#encoder_outputs, forward_h, forward_c, backward_h, backward_c=Bidirectional(LSTM( 200 , return_state=True ))( encoder_embedding )

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]



decoder_inputs = Input(shape=(None, ))
decoder_embedding = Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix) ,mask_zero=True) (decoder_inputs)

decoder_lstm = LSTM( 400 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense =Dense( VOCAB_SIZE , activation=keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    1430800     input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, None, 400),  641600      embedding_3[0][0]                
____________________________________________________________________________________________

In [9]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=15 ) 


Epoch 1/15
10000/10000 [==============================] - 137s 14ms/step - loss: 3.7066
Epoch 2/15
10000/10000 [==============================] - 136s 14ms/step - loss: 2.0040
Epoch 3/15
10000/10000 [==============================] - 135s 14ms/step - loss: 1.4816
Epoch 4/15
10000/10000 [==============================] - 134s 13ms/step - loss: 1.2633
Epoch 5/15
10000/10000 [==============================] - 135s 13ms/step - loss: 1.1343
Epoch 6/15
10000/10000 [==============================] - 135s 13ms/step - loss: 1.0391
Epoch 7/15
10000/10000 [==============================] - 133s 13ms/step - loss: 0.9630
Epoch 8/15
10000/10000 [==============================] - 133s 13ms/step - loss: 0.8899
Epoch 9/15
10000/10000 [==============================] - 132s 13ms/step - loss: 0.8220
Epoch 10/15
10000/10000 [==============================] - 132s 13ms/step - loss: 0.7580
Epoch 11/15
10000/10000 [==============================] - 131s 13ms/step - loss: 0.6956
Epoch 12/15
10000/10000 [====

In [11]:
model.save( 'model-sequence-embedding-bidirectional.h5' ) 

In [185]:
import tensorflow as tf 
model = tf.keras.models.load_model('model-sequence-embedding-bidirectional.h5')

In [81]:
model.layers[1].embeddings*model.input[0]

<tf.Tensor 'mul_11:0' shape=(7154, 200) dtype=float32>

In [104]:
model.layers[7].input

[<tf.Tensor 'embedding_2_3/embedding_lookup/Identity_1:0' shape=(?, ?, 200) dtype=float32>,
 <tf.Tensor 'concatenate_1_8/concat:0' shape=(?, 400) dtype=float32>,
 <tf.Tensor 'concatenate_2_8/concat:0' shape=(?, 400) dtype=float32>]

In [121]:
model.layers

In [194]:
model.layers

In [133]:
decoder_state_input_h = Input(shape=( 400 ,))
decoder_state_input_c = Input(shape=( 400 ,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] 
decoder_lstm ( model.layers[7].input[0],initial_state=encoder_states )
 

[<tf.Tensor 'lstm_2_25/transpose_2:0' shape=(?, ?, 400) dtype=float32>,
 <tf.Tensor 'lstm_2_25/while/Exit_3:0' shape=(?, 400) dtype=float32>,
 <tf.Tensor 'lstm_2_25/while/Exit_4:0' shape=(?, 400) dtype=float32>]

In [247]:
from keras.layers import Concatenate,Input
from keras.models import Model
encoder_inputs = model.input[0]
encoder_embeddings=model.layers[1]



encoder_outputs, forward_h, forward_c, backward_h, backward_c=model.layers[3].output 
# state_h = [forward_h, backward_h]
# state_c = [forward_c, backward_c]

#encoder_embedding = Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix), mask_zero=True )
state_h=model.layers[5].output
state_c=model.layers[6].output
encoder_states = [state_h, state_c]
#k=model.layers[1].output*model.input[0]
k=model.input[0]*model.layers[1].input
encoder_model = Model(model.layers[1].embeddings,model.input[0], encoder_states)
decoder_embedding=model.layers[4].embeddings
decoder_inputs =model.input[1]
#decoder_outputs, state_h, state_c=model.layers[7].output 


decoder_state_input_h = Input(shape=( 400 ,))
decoder_state_input_c = Input(shape=( 400 ,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm=model.layers[7]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm (model.layers[7].input[0]*model.input[1],initial_state=encoder_states )
 
decoder_states = [state_h_dec, state_c_dec]



decoder_dense=model.layers[8]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model = Model( [decoder_outputs] + decoder_states)

In [268]:
model.layers[6].output

<tf.Tensor 'concatenate_6/concat:0' shape=(?, 400) dtype=float32>

In [273]:
[decoder_inputs]

[<tf.Tensor 'input_97:0' shape=(?, ?) dtype=float32>]

In [10]:

def make_inference_models():
    
    encoder_model = Model(encoder_inputs, encoder_states)
    decoder_embedding = Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix) ,mask_zero=True) (decoder_inputs)


    decoder_state_input_h = Input(shape=( 400 ,))
    decoder_state_input_c = Input(shape=( 400 ,))
 
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [14]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [12]:
enc_model , dec_model = make_inference_models()

In [15]:

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : how long will it last
 i don't know it it end
Enter question : who are you
 no no they are a this you you and it end
Enter question : how long is the cord
 about about about 6 feet end
Enter question : is the phone good
 no it is not you to use the phone end
Enter question : does it have a flip
 no there no a remote end
Enter question : is the screen big enough
 yes i can it at at the and it end
Enter question : how long is it
 it it is the at at the at at at at end
Enter question : is the electronics good
 yes i can it at at at at at at at at end
Enter question : how long for charge
 the the and it was buy the picture end
Enter question : is camera good
 yes it is and the film end
